In [ ]:
!pip install tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [5]:
# Dados de exemplo (substitua pelos seus dados de clientes)
data = np.random.rand(1000, 10)  # 1000 clientes, 10 características

In [6]:
# Definir o gerador
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=100),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(data.shape[1], activation='sigmoid')
    ])
    return model

# Definir o discriminador
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', input_shape=(data.shape[1],)),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Compilar os modelos
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Definir a GAN
discriminator.trainable = False
gan_input = layers.Input(shape=(100,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Treinamento da GAN
def train_gan(gan, generator, discriminator, data, epochs=10000, batch_size=128):
    for epoch in range(epochs):
        # Treinar o discriminador
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_data = generator.predict(noise)
        real_data = data[np.random.randint(0, data.shape[0], batch_size)]
        combined_data = np.concatenate([real_data, generated_data])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        d_loss = discriminator.train_on_batch(combined_data, labels)

        # Treinar o gerador
        noise = np.random.normal(0, 1, (batch_size, 100))
        misleading_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)

        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

/Users/gabrielsantos/Programming/Python/DS-useful-codes/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Treinar a GAN
#train_gan(gan, generator, discriminator, data)

In [9]:
# Gerar dados sintéticos
noise = np.random.normal(0, 1, (1000, 100))
synthetic_data = generator.predict(noise)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [10]:
synthetic_data

array([[0.4008165 , 0.5800118 , 0.54340154, ..., 0.26501888, 0.32330608,
        0.47051   ],
       [0.53604656, 0.5849404 , 0.5818162 , ..., 0.4545664 , 0.42251897,
        0.4944078 ],
       [0.4389861 , 0.6372395 , 0.45832837, ..., 0.33542344, 0.32679877,
        0.5543906 ],
       ...,
       [0.46737742, 0.54867464, 0.57548684, ..., 0.36761215, 0.3397899 ,
        0.5451847 ],
       [0.45582876, 0.5680332 , 0.4897923 , ..., 0.43130422, 0.4025774 ,
        0.520517  ],
       [0.42117012, 0.5261151 , 0.42199954, ..., 0.4364507 , 0.36540106,
        0.53169656]], dtype=float32)